In [1]:
import bs4
import pandas as pd
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse

# 드라이버 위치 설정
driver_loc = "../chromedriver.exe"
# 드라이버 옵션 설정

options = webdriver.ChromeOptions()

options.add_argument("window-size=1920x1080") # 파이썬이 크롬창을 띄울 것인데 창 크기 선택
# 리눅스처럼 웹 화면 없는 경우에도 실행 가능.
# options.add_argument('headless')                           
# options.add_argument('disable-gpu')
# options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

# 웹 드라이버 정의
driver = webdriver.Chrome(driver_loc, options=options)

# 에누리 웹페이지 이동
# 탐색할 URL 정의
searchUrl = 'http://www.enuri.com/'

driver.get(searchUrl)

driver.maximize_window()



categoryMenu = driver.find_element_by_xpath('//*[@id="gnbMenu"]/li[4]/a/em')
categoryMenu.click()

# 스마트워치 xpath
tag_main =driver.find_element_by_xpath('//*[@id="first_depth3"]/li[7]/p') 
tag_sub = driver.find_element_by_xpath('//*[@id="first_depth3"]/li[7]/div/ul[2]/li[1]/a')

ActionChains(driver).move_to_element(tag_main).click(tag_sub).perform()
time.sleep(1.5)

# 가격비교 이동
compareXpath = '//*[@id="tabsDiv"]/li[2]/a'
compareMenu = driver.find_element_by_xpath(compareXpath)
compareMenu.click()
time.sleep(0.5)

# 90개씩 보기
viewXpath = '//*[@id="pageGapSelDiv"]/a'
viewMenu = driver.find_element_by_xpath(viewXpath)
viewMenu.click()
time.sleep(0.5)

ninetyXpath = '//*[@id="pageGapSelDiv"]/ul/li[3]/a'
ninetyMenu = driver.find_element_by_xpath(ninetyXpath)
ninetyMenu.click()

rowList = []
titleList = []
titleLenList = []
nameList = []

pageUrl = driver.current_url
print(pageUrl)
# 'http://www.enuri.com/list.jsp?cate=03181105&tabType=1'
cateNum = urlparse(pageUrl).query.split("&")[0].split("=")[1]
print(cateNum)

count = 0
missCount = 0

for i in range(999):
    # 다음 페이지 이동
    pageNumUrl = pageUrl+'&page=' + str(i + 1)
    driver.get(pageNumUrl)
    time.sleep(5)
    
    flag = True
    notFoundCount = 0
    while flag:
        # 아이템 페이지 이동
        try:
            data = driver.page_source
            bs = bs4.BeautifulSoup(data, 'html.parser')

            ulTag = bs.find('ul', {'id':'listBodyDiv'})
        #         print(ulTag)
        #         print(len(ulTag))
            ulTag.text
            liTags = ulTag.findAll('li', {'cate':cateNum})

            flag = False
        except:
            notFoundCount += 1
            print("Not Found", notFoundCount)
            print("reSearch!!")
            time.sleep(1)
            if notFoundCount > 6:
                break
            continue
            
    if (len(liTags) == 0) :
        break
            
    for j in range(len(liTags)):
        try:
            passCertain = liTags[j].findAll('div',{'class':'summary'})[1].find('b')
        except:
            missCount += 1
            print("스마트워치 아님 제외!!", liTags[j], missCount)
            continue               

        itemId = liTags[j].attrs['id']
        itemXpath = '//*[@id="{}"]/div[1]/strong/a'.format(itemId)
#             print(j, " ", itemId)
        count += 1    

#             # 상품창으로 이동
        itemMenu = driver.find_element_by_xpath(itemXpath)
        itemMenu.click()

#             # 창이동
        driver.switch_to.window(driver.window_handles[-1])

                    
        searchFlag = True
        notFoundCount = 0
    
        while (searchFlag):
            try:
                # 아이템 페이지에서 테이블 긁어오기
                data = driver.page_source
                bs = bs4.BeautifulSoup(data, 'html.parser')
                divTag = bs.find("div", attrs={"class":"malllowprice"})
                div2Tag = divTag.find("div", {"class":"malllow__cont"})
                tableTag = div2Tag.find('table', {'class':'malllow__list'})
                tbodyTag = tableTag.find('tbody', {'id':'vip_malllow_item_list_id'})
                trTag = tbodyTag.find('tr')
                tdTag = trTag.find('td', {'class':'malllow__item price'})
#                 오류가 발생하지 않는다면 False를 받아내고 while문을 탈출
                searchFlag = False
            except:
                notFoundCount += 1
                print("Not Found", notFoundCount)
                print("reSearch!!")
                time.sleep(1)
                if notFoundCount > 6:
                    break
                continue
                
        # 한 상품의 컬럼명과 스펙정보를 담을 리스트 생성
        columnList = []
        titleColumnList = []
        
        # 최저가 및 최저가 컬럼명 추출
        lowPriceTitle = tdTag.find('span').text
        titleColumnList.append(lowPriceTitle)
        lowPrice = tdTag.find('strong').text.replace(',','')
        columnList.append(lowPrice)

        # 상품명 추출
        titleNameDivTag = bs.find("div", {"class":"prdcinfo"})
        titleNameDiv2Tag = titleNameDivTag.find("div", {"class":"prdcinfo__tit"})
        titleNameH3Tag = titleNameDiv2Tag.find("h3", {"class":"prdcname"})
        titleName = titleNameH3Tag.text.strip().split("[")[0].replace(" ","")
        columnList.append(titleName)
        titleColumnList.append("상품명")
        
        
        # 제조사 추출
        makerNameDivTag = bs.find("div", {"id":"prdinfo_vip_attribute"})
        makerNameDiv2Tag = makerNameDivTag.find("div", {"class":"summary"})
        makerNameList = makerNameDiv2Tag.text.strip().split("|")[0].replace(" ","").split(":")
        
        if len(makerNameList) > 0:
            makerNameTitle = makerNameList[0]
            
            if makerNameTitle == "제조사":
                makerName = makerNameList[1]
                columnList.append(makerName)
                titleColumnList.append(makerNameTitle)

        # 상품 태그 정보 추출
        div3Tag = bs.find('div', {'id':'enuri_spec_table'})
        table2Tag = div3Tag.findAll('table', {'class':'offerer__table'})
        
        for k in range(len(table2Tag)):
            if None != table2Tag[k]:
                tbody2Tag = table2Tag[k].find('tbody')
                thTags = tbody2Tag.findAll('th', {'scope':'row'})
                tdTags = tbody2Tag.findAll('td')
                for l in range(len(tdTags)):
                    if thTags[l].text.strip() != '':
                        itemCount = titleColumnList.count(thTags[l].text.strip())
                        if itemCount > 0:
                            titleColumnList.append(thTags[l].text.strip() + str(itemCount + 1))
                        else:
                            titleColumnList.append(thTags[l].text.strip())

                        columnList.append(tdTags[l].text.strip().replace('\t',''))

        print(columnList)
        titleList.append(titleColumnList)
        titleLenList.append(len(titleColumnList))
        rowList.append(columnList)
        print(len(rowList))

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

titleListLen = len(titleList)

# 기준컬럼을 만든다
for i in range(titleListLen):
    for j in range(len(titleList[i])):
        if not titleList[i][j] in titleList[0]:
            titleList[0].append(titleList[i][j])
            rowList[0].append('')
                
# 기준컬럼리스트에 맞게 모든 상품의 컬럼리스트를 동일하게 만든다.
for i in range(titleListLen):
    for j in range(len(titleList[0])):
        # 중복요소 고려!!
        if not titleList[0][j] in titleList[i]:
            titleList[i].append(titleList[0][j])
            rowList[i].append('')

# 컬럼리스트를 정렬한다.
for i in range(titleListLen):
    for j in range(len(titleList[0])):
        if titleList[0][j] != titleList[i][j]:
            tmpTitleIndex = titleList[i].index(titleList[0][j]) # index 5
            tmpTitle = titleList[i][j] # 4
            titleList[i][j] = titleList[0][j] # 1
            titleList[i][tmpTitleIndex] = tmpTitle # 4

            tmpIndex = tmpTitleIndex
            tmp = rowList[i][j]
            rowList[i][j] = rowList[0][j]
            rowList[i][tmpIndex] = tmp

smartWatchEnuriData = pd.DataFrame(rowList, columns=titleList[0])
smartWatchEnuriData.to_csv('./smartWatch(enuri)_ver(1.0).csv', index = False , encoding = 'ms949')


http://www.enuri.com/list.jsp?cate=03181105&tabType=1
03181105
['316520', '삼성전자갤럭시워치액티브244mm스테인리스', '삼성전자', '스마트워치', '밴드교체형', '무선랜(WiFi)', '1.4"', '360x360', '슈퍼AM-OLED', '터치스크린', '블루투스5.0', '타이젠', '안드로이드5.0, iOS9.0이상', '768M', '4G내장', 'SNS, 문자, 전화', '스트레스모니터링, 이동거리측정, 달리기, 사이클링, 수영모니터링, 심박측정, 칼로리측정, 운동정보기록, 수면패턴분석, 만보계', '다중언어, GPS, 원격촬영, 음성명령, 화면테마설정, 자이로센서, NFC, 무선충전', '생활방수(IP68)', '평균 60시간,최대 131시간', '배터리일체형', '330mAh', '스테인리스스틸', '42g']
1
['192750', '삼성전자갤럭시워치액티브240mm알루미늄', '삼성전자', '스마트워치', '무선랜(WiFi)', '1.2"', '360x360', '슈퍼AM-OLED', '터치스크린', '블루투스5.0', '타이젠', '안드로이드5.0, iOS5.0이상', '768M', '4G내장', 'SNS, 문자, 전화', '스트레스모니터링, 달리기, 사이클링, 수영모니터링, 심박측정, 칼로리측정, 운동정보기록, 수면패턴분석, 만보계', '원격촬영, 음성명령, 화면테마설정', '평균 43시간,최대 95시간', '배터리일체형', '247mAh', '알루미늄', '26g']
2
['235180', '애플워치시리즈338mm(실버알루미늄)', '애플', '스마트워치', '밴드교체형', '무선랜(WiFi)', '전화통화', '블루투스', 'iOS', '듀얼코어', '8G내장', '일정, 문자, 전화', '스트레스모니터링, 일일활동량측정, 이동거리측정, 달리기, 심박측정, 칼로리측정, 운동정보기록, 운동코칭', '심박센서, GPS, SOS안심서비스, 고도센서, 기압센서', '방수(최대50m

AttributeError: 'NoneType' object has no attribute 'findAll'